In [7]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
import scipy.io
from sklearn.model_selection import train_test_split

In [4]:
mnist = scipy.io.loadmat('mnist_data.mat')
mnist_training = mnist["training_data"]
mnist_labels = mnist["training_labels"]
mnist_test = mnist["test_data"]

In [10]:
class GDA:
    """Perform Gaussian discriminant analysis (both LDA and QDA)."""
    def __init__(self, *args, **kwargs):
        self._fit = False
        
        #TODO: Possibly add new instance variables
        self.classes = None 
        self.pooled_cov = None
        self.params = {}

    def evaluate(self, X, y, alpha=1, mode="lda"):
        """Predict and evaluate the accuracy using zero-one loss.

        Args:
            X (np.ndarray): The feature matrix shape (n, d)
            y (np.ndarray): The true labels shape (d,)

        Optional:
            mode (str): Either "lda" or "qda".

        Returns:
            float: The accuracy loss of the learner.

        Raises:
            RuntimeError: If an unknown mode is passed into the method.
        """
        #TODO: Compute predictions of trained model and calculate accuracy
        #Hint: call `predict` to simplify logic
        pred = self.predict(X, alpha, mode=mode)
        accuracy = np.sum(pred == y.flatten()) / y.flatten().shape[0]
        return accuracy
    
    def error_rate(self, X, y, alpha=1, mode="lda"):
        return 1 - self.evaluate(X, y, alpha, mode=mode)

    def fit(self, X, y):
        """Train the GDA model (both LDA and QDA).

        Args:
            X (np.ndarray): The feature matrix (n, d)
            y (np.ndarray): The true labels (n, d)
        """
        #TODO: Train both the QDA and LDA model params based on the training data passed in
        # This will most likely involve setting instance variables that can be accessed at test time
        
        # the model is fitted 
        self._fit = True
        # set classes 
        self.classes = np.unique(y)
        n = X.shape[0]
        d = X.shape[1]
        cov_var = np.zeros((d, d))
        self.params = {}
        for c in self.classes: 
            # current training data 
            indices = (y.flatten() == c)
            cur_training = X[indices]
            num_data = cur_training.shape[0]
            # compute the prior 
            prior = num_data / n 
            # compute the class mean 
            ave = np.mean(cur_training, axis=0)
            # compute the pooled within-class var 
            cur_var = (cur_training - ave).T @ (cur_training - ave)
            cov_var = cov_var + cur_var
            self.params[c] = (prior, ave, cur_var / num_data)
        self.pooled_cov = cov_var / n
        

    def predict(self, X, alpha=1, mode="lda"):
        """Use the fitted model to make predictions.

        Args:
            X (np.ndarray): The feature matrix of shape (n, d)
            alpha: The constant to be added to the covariance matrix 

        Optional:
            mode (str): Either "lda" or "qda".

        Returns:
            np.ndarray: The array of predictions of shape (n,)

        Raises:
            RuntimeError: If an unknown mode is passed into the method.
            RuntimeError: If called before model is trained
        """
        if not self._fit:
            raise RuntimeError("Cannot predict for a model before `fit` is called")

        preds = None
        if mode == "lda":
            #TODO: Compute test-time preditions for LDA model trained in 'fit'
#             log_posteriors = []
#             for c in np.sort(self.classes):
#                 log_prior = np.log(self.params[c][0])
#                 log_likelihood = multivariate_normal.logpdf(x=X, 
#                                                             mean=self.params[c][1], 
#                                                             cov=self.pooled_cov, 
#                                                             allow_singular=True)
#                 log_posterior = log_likelihood + log_prior 
#                 log_posteriors.append(log_posterior)
#             preds = np.argmax(log_posteriors, axis=0)
            mod_cov = self.pooled_cov + alpha * np.identity(self.pooled_cov.shape[0])
            deltas = []
            for c in self.classes:
                prior = self.params[c][0]
                mean = self.params[c][1]
                inverse = np.linalg.inv(mod_cov)
                delta = X @ inverse @ mean - 0.5 * mean.T @ inverse @ mean + np.log(prior) 
                deltas.append(delta)
            preds = self.classes[np.argmax(deltas, axis=0)]
        elif mode == "qda":
            log_posteriors = []
            for c in self.classes:
                log_prior = np.log(self.params[c][0])
                log_likelihood = multivariate_normal.logpdf(x=X, 
                                                            mean=self.params[c][1], 
                                                            cov=self.params[c][2], 
                                                            allow_singular=True)
                log_posterior = log_likelihood + log_prior 
                log_posteriors.append(log_posterior)
            preds = self.classes[np.argmax(log_posteriors, axis=0)]
#             deltas = [] 
#             for c in np.sort(self.classes):
#                 cov = self.params[c][2]
#                 mod_cov = cov + alpha * np.identity(cov.shape[0])
#                 prior = self.params[c][0]
#                 mean = self.params[c][1]
#                 inverse = np.linalg.inv(mod_cov)
#                 delta = -0.5*X@inverse@X.T + X@inverse@mean - 0.5*mean.T@inverse@mean + np.log(prior)
#                 deltas.append(delta)
#             preds = np.argmax(deltas, axis=0)
        else:
            raise RuntimeError("Unknown mode!")
        return preds

In [6]:
def k_fold_cv(X, y, k, alpha=1, mode="lda"):
    acc = [] 
    # list of arrays of indices 
    indices = np.array_split(np.arange(X.shape[0]), k)
    for i in tqdm(range(len(indices))):
        # copy 
        a = indices.copy()
        train_indices = np.concatenate(a)
        test_indices = a.pop(i)
        X_train = X[train_indices, :]
        y_train = y[train_indices, :]
        X_test = X[test_indices, :]
        y_test = y[test_indices, :]
        model = GDA()
        model.fit(X_train, y_train)
        acc.append(model.evaluate(X_test, y_test, alpha, mode=mode))
    return np.mean(acc)

In [8]:
def extract_hog(X, orientations, pixels_per_cell, cells_per_block, resizeFactor, anti_aliasing):
    new_features = []
    for i in tqdm(range(X.shape[0])):
        cur_image = X[i].reshape(28, 28)
        resized = resize(cur_image, resizeFactor, anti_aliasing=anti_aliasing)
        edge = filters.sobel(resized)
        cur_features = hog(edge, orientations=orientations, pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block, visualize=False, multichannel=False)
        new_features.append(cur_features)
    new_features = np.asarray(new_features)
    return new_features

In [11]:
cv = k_fold_cv(mnist_training, mnist_labels, 5, alpha=1, mode="lda")
print("5-fold CV score for LDA is: {}".format(cv))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.46s/it]

5-fold CV score for LDA is: 0.8712500000000001


# Using SVM

In [ ]:
from sklearn.svm import SVC


# Using NN